In [ ]:
#  06_prophet_trial_01.ipynb
#  prophetによる予測プログラムのパラメータ改善　20221130
#  Optunaが時間がかかるので手作業による最適値捜索
#  変化させるパラメータは'changepoint_prior_scale'のみ
#  こちらが本番。06_prophet_trial_00_testの結果から'changepoint_prior_scale'は0.001から0.01まで0.001間隔
#  出力値は最良のRMSEを得た場合のcsvとpngとRMSEとRMSE/停止水蒸気量
#  20230106 計算時間がかかりすぎるためパラメータ変更：30分間隔出力から１日間隔出力に変更。また、cps = 0.005に固定

#  20230113 1日から10日までRMSEが出るように仕様変更
#  20230113 パラメータディフォルト値の場合のRMSEと比較追加

#  prophetによる蒸気発生量予測：　20221102

#  大藤手入れ　2023-5-19
#https://qiita.com/dcm_shimadas/items/8f049c4a258f0fbbfe23  <-  optuna参考ページ
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/forecast/optuna_2')

from sklearn.model_selection import TimeSeriesSplit
import glob
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error


In [ ]:
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
%matplotlib inline

In [ ]:
# Prophetの出力をミュート
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)

In [ ]:
#取り出す期間
#予測失敗期間　20080204163500(06T)      20080409160500(06T)  20160820205000(15T)
#予測が大きく変わった期間 20160522170500(06T)     2010121192000(14T)    20101228192000(14T)  20160501205000 -> 20160502205000(15T)
#20160606205000 -> 20160607205000(15T)     20160609205000 -> 20160610205000(15T)
#ジャンプがある期間　　2008022163500(06T)    20160126170500(06T)   20160522170500(06T)
#学習失敗期間(学習が追えていない) 20160820205000->20160821205000->20160822205000(15T)       20200501145500(15T)
#この辺の期間を取り出して時間粒度やパラメータを変えて予測精度を上げていきたい。
#15Tの2016年の5月あたりがを取り出して色々試してみるのがいいかも (15T_20151113_20170411.csvとかでやってみる)
#20160802195500から20160924195500まではジャンプが大きいからこの期間の取り出しもあり。

In [ ]:
#ファイル読み込み
data_tmp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/forecast/001_segment_data_stop_judge/15T_20151113_20170411.csv',
                    index_col = 'date_time',
                    parse_dates = True)

In [ ]:
print(data_tmp.index.get_loc('2016-08-02 19:55:00'))
print(data_tmp.index.get_loc('2016-09-24 19:55:00'))

75745
91009


In [ ]:
data_tmp.index.get_loc('2016-08-04 19:55:00') - data_tmp.index.get_loc('2016-08-03 19:55:00')

288

In [ ]:
# 変数の定義 学習粒度と予測対象期間は色々値を変えてみる(リストにしてループで回すのが良さそう)
learning_frequency = 60 #学習粒度(分) 5と30は実施済み [5,10,30,60,120]
learning_time = 20 # 学習期間　20日
prediction_time = 7 # 予測対象期間　7日 ここも値を変える

In [ ]:
day_number_list = [1]

In [ ]:
for day_number in day_number_list:
    well = '15T_20151113_20170411'
    os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_csv01/' , exist_ok=True)
    os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_png01/' , exist_ok=True)
    os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_rmse01/' , exist_ok=True)
    os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_parameter01/' , exist_ok=True)

    # 停止水蒸気量算出
    start_time        = data_tmp.index[75745+288*day_number]
    end_time          = start_time + dt.timedelta(days = 20)
    data_for_stop     = data_tmp[start_time:end_time]
    steamvol_for_stop = data_for_stop['steamvol'].mean()*(2/3)
    end_data          = data_tmp.index[91009]
    data_start_time   = start_time.strftime('%Y%m%d%H%M%S')
    data_end_time     = end_data.strftime('%Y%m%d%H%M%S')

    # 出力フォルダ作成 # このへんmkdirs(, exist_ok = True) あたりを使用するとすっきりする（途中ディレクトリも作ってくれる）
    #os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_csv01/'+ well + '/' + data_start_time + '_' + data_end_time , exist_ok=True)
    #os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_png01/'+ well + '/' + data_start_time + '_' + data_end_time , exist_ok=True)
    #os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_rmse01/'+ well + '/' + data_start_time + '_' + data_end_time , exist_ok=True)
    #os.makedirs('learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_parameter01/'+ well + '/' + data_start_time + '_' + data_end_time , exist_ok=True)

    #pathをとっておく
    csv_path = 'learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_csv01/'
    png_path = 'learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_png01/'
    rmse_path = 'learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_rmse01/'
    parameter_path = 'learn_freq='+str(learning_frequency)+', prediction='+str(prediction_time)+'/008_pred_parameter01/'

    # 学習、予測の日付時刻の取得
    start_date_time = data_tmp.index[75745+288*day_number] # 学習開始
    end_date_time = start_date_time + dt.timedelta(days = learning_time) # 学習終了
    prediction_start_date_time = end_date_time + dt.timedelta(minutes = learning_frequency) # 予測開始（学習が終了した後学習粒度の分だけ経ったら開始）
    prediction_end_date_time = prediction_start_date_time + dt.timedelta(days = prediction_time) # 予測終了

    # 24時間ごとにシフトさせて予測
    prediction_start_date_time_list = []
    rmse_list = []
    thifted_time = 24
    counter = 1

    start_date_time = data_tmp.index[75745+288*day_number]
    end_date_time = start_date_time + dt.timedelta(days = learning_time)
    start_date_time_str = str(start_date_time.strftime('%Y%m%d%H%M%S'))
    end_date_time_str = str(end_date_time.strftime('%Y%m%d%H%M%S'))

    # 予測期間
    prediction_start_date_time = end_date_time + dt.timedelta(minutes = learning_frequency)
    prediction_end_date_time = prediction_start_date_time + dt.timedelta(days = prediction_time)

    prediction_start_date_time_str = str(prediction_start_date_time.strftime('%Y%m%d%H%M%S'))
    prediction_end_date_time_str   = str(prediction_end_date_time.strftime('%Y%m%d%H%M%S'))
    print(" ** ", well, " の ", start_date_time_str, " から ", end_date_time_str, " までを学習します ** ")
    print(" ** ", well, " の ", prediction_start_date_time_str, " から ", prediction_end_date_time_str, " までを予測します ** ")

    # 学習期間→train, 予測期間→test
    train = data_tmp[start_date_time:end_date_time]
    train = train.asfreq(str(learning_frequency)+'min') #学習粒度でリサンプリング
    test = data_tmp[prediction_start_date_time:prediction_end_date_time]
    test_length = len(test)

    train.reset_index(inplace = True)
    test.reset_index(inplace = True)

    train = train.rename(columns = {'date_time':'ds','steamvol':'y'})
    test  = test.rename( columns = {'date_time':'ds','steamvol':'y'})


    #
# Prophetによる予測
#ここからパラメータのチューニング
#ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
    def objective(trial):
        params = {
            #'uncertainty_samples' : trial.suggest_int('uncertainty_samples', 100, 1000, 450),
            'changepoint_range' : trial.suggest_discrete_uniform('changepoint_range',0.8,0.95,0.001),
            'n_changepoints' : trial.suggest_int('n_changepoints',20,35),
            'changepoint_prior_scale' : trial.suggest_discrete_uniform('changepoint_prior_scale',0.001,0.5,0.001),
            'seasonality_prior_scale' : trial.suggest_discrete_uniform('seasonality_prior_scale',1,25,0.5),
            'seasonality_mode' : trial.suggest_categorical('seasonality_mode',['additive', 'multiplicative'])
            }

        tss = TimeSeriesSplit(test_size = 5)

        cv_mse = []

        for fold, (train_index, valid_index) in enumerate(tss.split(train)):
            train_data = train.iloc[train_index] #訓練データ
            valid_data = train.iloc[valid_index] #検証データ

            m = Prophet(**params)

            m.fit(train_data)

            df_future = m.make_future_dataframe(
                periods = len(valid_data),
                freq = str(learning_frequency)+'min'
            )

            df_pred = m.predict(df_future)
            preds = df_pred.tail(len(valid_data))

            val_mse = mean_squared_error(
                valid_data.y,
                preds.yhat
            )
            cv_mse.append(val_mse)

        return np.mean(cv_mse)

    optuna.logging.disable_default_handler()

    study = optuna.create_study(direction = 'minimize')
    #n_trialsの値は1000でいいのか
    study.optimize(
        objective,
        n_trials = 1000,
        n_jobs = -1
        )



    #ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

    #データフレームを作る(ベストパラメータを保存するため)
    parameter_list = pd.DataFrame()
    parameter_list['best_parameter'] = study.best_params
    parameter_list.to_csv(parameter_path + prediction_start_date_time_str + '.csv')

    model_best = Prophet(**study.best_params)
    model_best.fit(train)
    future = model_best.make_future_dataframe(periods = test_length, freq = str(learning_frequency)+'min')
    pred_best  =  model_best.predict(future)
    pd.to_datetime(pred_best['ds'])
    pred_best = pred_best.set_index('ds')

    merged_data_best = pd.merge(data_tmp,
                                pred_best,
                                right_index = True,
                                left_index = True,
                                how = 'inner')

    merged_data_best = merged_data_best[start_date_time:prediction_end_date_time] # 学習期間も含めた
    merged_data_best.to_csv(csv_path + prediction_start_date_time_str + '.csv')



    #
    # RMSE算出
    #

    # 2日後までの 総計RMSE 算出
    actual_best = merged_data_best['steamvol']
    pred_data_best = merged_data_best['yhat']
    RMSE_best = np.sqrt(mean_squared_error(actual_best[0:47], pred_data_best[0:47]))

    print(" ** 2日間のRMSEは、", round(RMSE_best, 3), " でした **  ")

    # 1日から7日までの 各日のRMSE 算出
    days_list = ['1day','2days','3days','4days','5days','6days','7days']
    rmse_list = []
    for i in [1,2,3,4,5,6,7]:
        start = merged_data_best.index[0] + dt.timedelta(days = 1*(i-1))
        end = start + dt.timedelta(days = 1) + dt.timedelta(minutes = learning_frequency)
        actual_tmp = merged_data_best['steamvol'][start:end]
        pred_data_tmp = merged_data_best['yhat'][start:end]
        try:
            rmse_tmp = np.sqrt(mean_squared_error(actual_tmp, pred_data_tmp))
        except:
            rmse_tmp = np.nan
        rmse_list.append(rmse_tmp)

    '''
    # 比較のためのデフォルト値での予測
    model_default = Prophet(changepoint_prior_scale = 0.5)
    model_default.fit(train)
    future_default  =  model_default.make_future_dataframe(periods = test_length, freq = '60min')
    pred_default  =  model_default.predict(future_default)
    pd.to_datetime(pred_default['ds'])
    pred_default = pred_default.set_index('ds')
    merged_data_default = pd.merge(data_tmp,pred_default,right_index = True,left_index = True,how = 'inner')
    merged_data_default = merged_data_default[prediction_start_date_time:prediction_end_date_time]
    actual_00 = merged_data_default['steamvol']
    pred_data_00 = merged_data_default['yhat']
    RMSE_00 = np.sqrt(mean_squared_error(actual_00[0:47], pred_data_00[0:47]))# 一応算出：改善度合いをみるため
    '''

    #
    # プロット
    #
    plt.rcParams['figure.figsize'] = (20,5)

    plt.plot(merged_data_best['steamvol'], # 実測値
            color = 'black',
            label = 'actual_data')

    plt.plot(merged_data_best.loc[start_date_time:end_date_time,
                                'yhat'], # 学習期間
            color = 'cyan',
            label = 'Training ')

    plt.plot(merged_data_best.loc[prediction_start_date_time:prediction_end_date_time,
                                'yhat'], # 予測期間
            color = 'blue',
            label = 'Prediction: best RMSE = ' + str(round(RMSE_best, 5)))

    plt.axvline(prediction_start_date_time,
                color='blue', linestyle='--')

    # plt.plot(merged_data_default['yhat'],color = 'red',label = 'default RMSE = ' + str(RMSE_00)) # デフォルト（cps=0.5）プロット

    plt.legend()
    plt.grid()
    plt.title(str(well)+'  learning_freq = '+str(learning_frequency)+ 'minutes    prediction_time = ' +str(prediction_time)+'days', loc = "left")

    plt.savefig(png_path + prediction_start_date_time_str + '.png')
    plt.clf()

    print(" ** plot : ", png_path + prediction_start_date_time_str , " .png を書き出しました ** ")


    # RMSE出力
    df_rmse = pd.DataFrame()
    df_rmse['days'] = days_list
    df_rmse['RMSE'] = rmse_list
    df_rmse['normed_RMSE'] = rmse_list/steamvol_for_stop
    df_rmse = df_rmse.set_index('days')
    df_rmse = df_rmse.T
    df_rmse['RMSE_best_total'] = [RMSE_best,RMSE_best/steamvol_for_stop]
    # df_rmse['RMSE_default_total'] = [RMSE_00,RMSE_00/steamvol_for_stop]
    df_rmse = df_rmse.T

    df_rmse.to_csv(rmse_path + prediction_start_date_time_str + '.csv')


    counter = counter + 1
    print("============================================")


 **  15T_20151113_20170411  の  20160804195500  から  20160824195500  までを学習します ** 
 **  15T_20151113_20170411  の  20160824205500  から  20160831205500  までを予測します ** 


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Custom TB Handler failed, unregistering
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-3f69a0d84448>", line 127, in <cell line: 1>
    parameter_list.to_csv(parameter_path + prediction_start_date_time_str + '.csv')
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 3720, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py", line 1189, in to_csv
    csv_formatter.save()
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/formats/csvs.py", line 241, in save
    with get_handle(
  File "

In [ ]:
print(study.best_params)

{'changepoint_range': 0.8580000000000001, 'n_changepoints': 25, 'changepoint_prior_scale': 0.057, 'seasonality_prior_scale': 18.5, 'seasonality_mode': 'additive'}


 ** 2日間のRMSEは、 0.208  でした **  
 ** plot :  learn_freq=60, prediction=7/008_pred_png01/15T_20151113_20170411/20160806195500_20160924195500/20160826205500  .png を書き出しました ** 


<Figure size 2000x500 with 0 Axes>

In [ ]:
def objective_variable(train,valid):
  def objective(trial):
            params = {
      'changepoint_range' : trial.suggest_float('changepoint_range',0.8,0.95, step = 0.001),
      'n_changepoints' : trial.suggest_int('n_changepoints',20,35),
      'changepoint_prior_scale' : trial.suggest_float('changepoint_prior_scale',0.001,0.5),
      'seasonality_prior_scale' : trial.suggest_float('seasonality_prior_scale',0.01,10),
      'seasonality_mode' : trial.suggest_categorical('seasonality_mode',['additive', 'multiplicative']),
        }
            # fit_model
            m=Prophet(**params)
            m.fit(train)
            future = m.make_future_dataframe(periods=len(valid))

            forecast = m.predict(future)
            valid_forecast = forecast.tail(len(valid))

            val_mape = np.mean(np.abs((valid_forecast.yhat-valid.y)/valid.y))*100
            return val_mape
  return objective



In [ ]:
def optuna_parameter(train,valid):
    study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=42))
    study.optimize(objective_variable(train,valid), timeout=1000)
    optuna_best_params = study.best_params

    return study